# Data Crawling
## Install and import packages

In [1]:
#!pip install vnstock pandas requests bs4

  Using cached vnstock-0.2.8.7-py3-none-any.whl (36 kB)
  Using cached pandas-2.1.4-cp311-cp311-macosx_10_9_x86_64.whl (11.6 MB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached bs4-0.0.1-py3-none-any.whl
  Using cached numpy-1.26.2-cp311-cp311-macosx_10_9_x86_64.whl (20.6 MB)
  Using cached pytz-2023.3.post1-py2.py3-none-any.whl (502 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Using cached charset_normalizer-3.3.2-cp311-cp311-macosx_10_9_x86_64.whl (121 kB)
  Using cached idna-3.6-py3-none-any.whl (61 kB)
  Using cached urllib3-2.1.0-py3-none-any.whl (104 kB)
  Using cached certifi-2023.11.17-py3-none-any.whl (162 kB)
  Using cached beautifulsoup4-4.12.2-py3-none-any.whl (142 kB)
  Using cached soupsieve-2.5-py3-none-any.whl (36 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
from vnstock import *

In [5]:
listing_companies(live=True)

,ticker,organName,organTypeCode,comGroupCode
0,A32,CTCP 32,1,UPCOM
1,AAA,CTCP Nhựa An Phát Xanh,1,HOSE
2,AAM,CTCP Thủy sản MeKong,1,HOSE
3,AAS,CTCP Chứng khoán SmartInvest,4,UPCOM
4,AAT,CTCP Tập Đoàn Tiên Sơn Thanh Hóa,1,HOSE
...,...,...,...,...
1581,XPH,CTCP Xà phòng Hà Nội,1,UPCOM
1582,YBC,CTCP Xi măng và Khoáng sản Yên Bái,1,UPCOM
1583,YBM,CTCP Khoáng sản Công nghiệp Yên Bái,1,HOSE
1584,YEG,CTCP Tập đoàn Yeah1,1,HOSE


In [7]:
indices_listing(lang='vi')

,comGroupCode,parentComGroupCode,comGroupOrder
0,VNINDEX,VNINDEX,1
1,HNXIndex,HNXIndex,2
2,VN30,VNINDEX,2
3,VNCOND,VNINDEX,2
4,HNX30,HNXIndex,3
5,UpcomIndex,UpcomIndex,3
6,VN100,VNINDEX,3
7,VNCONS,VNINDEX,3
8,VNENE,VNINDEX,4
9,VNX50,VNINDEX,4
